In [40]:
import librosa
import math
import numpy as np
import scipy.signal
from scipy.special import logsumexp
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, (1, 5))
        self.fc1 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 48)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sm(x)
        return x
    
def load_audio_to_melspec_tensor(wavpath, sample_rate=16000):
    window_size = .025
    window_stride = 0.01
    n_dft = 512
    win_length = int(sample_rate * window_size)
    hop_length = int(sample_rate * window_stride)
    y, sr = librosa.load(wavpath, sr=sample_rate)
    y = y - y.mean()
    y = np.append(y[0],y[1:]-.97*y[:-1])
    # compute mel spectrogram
    stft = librosa.stft(y, n_fft=n_dft, hop_length=hop_length,
        win_length=win_length, window=scipy.signal.hamming)
    spec = np.abs(stft)**2
    mel_basis = librosa.filters.mel(sr=sample_rate, n_fft=n_dft, n_mels=40, fmin=20)
    melspec = np.dot(mel_basis, spec)
    logspec = librosa.power_to_db(melspec, ref=np.max)
    logspec = np.transpose(logspec)
    logspec_tensor = torch.tensor(logspec)
    return logspec_tensor

def compute_phone_likelihoods(model, logspec):
    likelihood_list = []
    with torch.no_grad():
        for j in range(6, logspec.size(0) - 5):
            inp = logspec[j-5:j+6,:].unsqueeze(0)
            output = model(inp) # output will be log probabilities over classes
            output = output - math.log(1. / 48) # subtract the logprob of the class priors (assumed to be uniform)
            likelihood_list.append(output[0])
    likelihoods = torch.transpose(torch.stack(likelihood_list, dim=1), 0, 1).numpy()
    return likelihoods

class MyHMM:
    def __init__(self, state_labels, initial_state_distribution, transition_matrix, eps=1e-200):
        self.eps = eps
        self.pi = np.log(initial_state_distribution + eps)
        self.A = np.log(transition_matrix + eps) #A_{ji} is prob of transitioning from state j to state i
        self.labels = state_labels # a list where self.labels[j] is the index of the phone label belonging to the jth state
        print(self.labels)
        self.N_states = len(self.labels)

    def forward(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        N_timesteps, _ = state_likelihoods.shape
        #print(N_timesteps)
        
        # Initialization
        log_alpha =[]
        for i in range(self.N_states):
            log_alpha.append(self.pi[i] * state_likelihoods[0, self.labels[i]])
        log_alpha = np.array(log_alpha)
        print(log_alpha)

        # Induction
        for t in range(1, N_timesteps):
            log_alpha_new = np.zeros(self.N_states)
            for i in range(self.N_states):
                #print(log_alpha,  log_A[:, i])
                log_sum = logsumexp(log_alpha * self.A[:, i])
                log_alpha_new[i] = log_sum * state_likelihoods[t, self.labels[i]]
            print(log_alpha)
            log_alpha = log_alpha_new
        
        # Termination
        #log_likelihood = logsumexp(log_alpha)
        return log_alpha[-1]    
 
    def viterbi(self, state_likelihoods):
        T = state_likelihoods.shape[0]
        delta = np.zeros((T, self.N_states))
        psi = np.zeros((T, self.N_states), dtype=int)
        delta[0] = self.pi + state_likelihoods[0, self.labels]

        for t in range(1, T):
            for j in range(self.N_states):
                temp = delta[t-1] + self.A[:,j] + state_likelihoods[t,self.labels]
                delta[t,j] = np.max(temp)
                psi[t,j] = np.argmax(temp)
            
        # termination
        q_star_T = np.argmax(delta[-1])

        #backtrace
        #asta
        q_star = [q_star_T]
        for t in range(T-1, 0, -1):
            q_star_t = psi[t,q_star[-1]]
            q_star.append(q_star_t)
        q_star.reverse()

        return q_star

    
    def viterbi_transition_update(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        # TODO: fill in
        pass

model = MyNet()
model.load_state_dict(torch.load('lab3_AM.pt'))

lab3_data = np.load('lab3_phone_labels.npz')
phone_labels = list(lab3_data['phone_labels'])
def phones2indices(phones):
    return [phone_labels.index(p) for p in phones]

fee_HMM = MyHMM(phones2indices(['sil', 'f', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
pea_HMM = MyHMM(phones2indices(['sil', 'p', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
rock_HMM = MyHMM(phones2indices(['sil', 'r', 'aa', 'cl', 'k', 'sil']), np.array([0.5,0.5,0,0,0,0]), np.array([[.9,.1,0,0,0,0],[0,.9,.1,0,0,0],[0,0,.9,.1,0,0],[0,0,0,.9,.1,0],[0,0,0,0,.9,.1],[0,0,0,0,0,1]]))
burt_HMM = MyHMM(phones2indices(['sil', 'b', 'er', 'cl', 't', 'sil']), np.array([0.5,0.5,0,0,0,0]), np.array([[.9,.1,0,0,0,0],[0,.9,.1,0,0,0],[0,0,.9,.1,0,0],[0,0,0,.9,.1,0],[0,0,0,0,.9,.1],[0,0,0,0,0,1]]))
see_HMM = MyHMM(phones2indices(['sil', 's', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
she_HMM = MyHMM(phones2indices(['sil', 'sh', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))

# TODO: write your code to use your HMMs below here (or in a new block)
fee_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor('fee.wav'))
print(fee_HMM.forward(fee_likelihoods))

pea_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor('fee.wav'))
print(pea_HMM.forward(pea_likelihoods))

rock_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor('fee.wav'))
print(rock_HMM.forward(rock_likelihoods))

burt_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor('fee.wav'))
print(burt_HMM.forward(burt_likelihoods))

[0, 43, 5, 0]
[0, 10, 5, 0]
[0, 4, 17, 9, 26, 0]
[0, 23, 30, 9, 37, 0]
[0, 1, 5, 0]
[0, 14, 5, 0]
[-2.58263075e+00  2.93913724e+00  4.54319089e+03 -1.71586273e+03]
[-2.58263075e+00  2.93913724e+00  4.54319089e+03 -1.71586273e+03]
[ 2.92529646e+06 -3.61989603e+06 -8.67460415e+06  4.40300991e+03]
[ 1.48784867e+10 -1.68747843e+10 -8.92113874e+07  6.20876459e+09]
[ 2.92242003e+13 -1.69928197e+11 -4.15648886e+11  2.92242003e+13]
[ 7.23294833e+14 -7.68776907e+14 -4.03112407e+12  2.95701952e+14]
[ 1.34087446e+18 -5.55123855e+15 -1.63040527e+16  1.34087446e+18]
[ 2.83305484e+19 -2.60669458e+19 -1.18647098e+17  9.64604540e+18]
[ 4.50338855e+22 -2.19635106e+20 -6.16114310e+20  4.50338855e+22]
[ 1.06773394e+24 -1.21887699e+24 -5.40686128e+21  3.80630433e+23]
[ 2.13904171e+27 -1.29675320e+25 -3.26090355e+25  2.13904171e+27]
[ 5.74284751e+28 -7.52042397e+28 -3.42420632e+26  2.28374001e+28]
[ 1.31692831e+32 -6.27229560e+29 -1.73769077e+30  1.31692831e+32]
[ 3.04836327e+33 -2.73796320e+33 -1.30484739

<ipython-input-40-961f074e946b>:93: RuntimeWarning: overflow encountered in multiply
  log_sum = logsumexp(log_alpha * self.A[:, i])
<ipython-input-40-961f074e946b>:93: RuntimeWarning: invalid value encountered in multiply
  log_sum = logsumexp(log_alpha * self.A[:, i])
